# MODEL PROJECT

In [6]:
import numpy as np
import scipy as sp
import math
from scipy import optimize
import sympy as sm
from scipy import linalg
from sympy import Derivative
from scipy import interpolate
import scipy.integrate as quad
from scipy.integrate import odeint
import scipy.special as special
from scipy.integrate import quad
from sympy import symbol,function
import ipywidgets as widgets

#autoreload model when code is run
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from matplotlib import cm
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

## Consumer optimization problem

Before moving to the Ak model adding the firm player to our model, we focus on consumers and their optimization problem. The standard Ak model assumes that the individual show a CRRA utility function with respect to consumption of the type:  

$$U(c)=\frac{c^{1-\theta}-1}{1-\theta}$$  

where $\theta$ is a measure or relative risk aversion
This type of preferences is really useful in the case $\theta=1$ because in that occasion the CRRA utility becomes a logarithmic utility function.  
Lets create a simple utility maximization problem with two periods to see how the above preferences impact the consumer behavior. We take into consideration two periods, $t=0$ and $t=1$.   
Our representative consumer maximizes consumption in the present $c_0$ and in the future $c_1$ taking into consideration his labour income will be $w_0$ today and $w_1$ tomorrow. So, our utility maximization problem become:  

$$max U(c_0,c_1)$$  

under the constraint:  

$$c_0+\frac{1}{1+r}{c_1} = w_0 +\frac{1}{1+r}{w_1}$$ 
where r is the interest rate.

Now, before solving the problem, we have to give some initial values:


In [7]:
#a.define initial parameters
theta = 0.5
rho = 0.9
r = 0.5
w_t = 5
w_t1 = 5

#b.define utility function
def utility(ct):
    return (ct**(1-theta)-1)/(1-theta)

We set the lagrangian:  

$$L=\max_{c_{0},c_{2}}\bigg(\frac{c_{0}^{1 - \theta} - 1}{1 - \theta}\bigg) + e^{-\rho}\bigg(\frac{c_{1}^{1 - \theta} - 1}{1 - \theta}\bigg) + \lambda\bigg(w_{0} + \frac{1}{1 + r}w_{1} - c_{0} - \frac{1}{1 + r}c_{1}\bigg)$$  

Focs are:
$$c_{0}^{-\theta} = \lambda$$
$$e^{-\rho}c_{1}^{-\theta}=\frac{1}{1 + r}\lambda$$  

The above can reduce to:
$$c_{0}^{-\theta} = e^{-\rho}(1 + r)c_{1}^{-\theta}$$  

We can now start writing our code:

In [9]:
#define utility function for two periods
def inter_util(c_t, c_t1):
    return utility(c_t)+np.exp(-rho)*utility(c_t1)

#b.define Euler equation
def euler(c_t,c_t1):
    return c_t**(-theta)-np.exp(-rho)*(1+r)*(c_t1)**(-theta)

#c.define constraint
def constraint(c_t,c_t1):
    return w_t+(1/(1+r))*w_t1-c_t-(1/(1+r))*c_t1

#d.creating optimizing function
def optimalchoice(x):
    op = [euler(x[0], x[1])]
    op.append(constraint(x[0],x[1]))
    return op

ct_guess, ct1_guess = 0.5, 0.5
ct_star, ct1_star = optimize.fsolve(func=optimalchoice, x0=(ct_guess, ct1_guess))
utility_star = inter_util(ct_star, ct1_star)

print(f'The optimal consumption at the present is: {ct_star:.2f}')
print(f'The optimal consumption tomorrow is: {ct1_star:.2f}')
print(f'Utility from optimal bundle is: {utility_star:.2f}')

The optimal consumption at the present is: 6.68
The optimal consumption tomorrow is: 2.48
Utility from optimal bundle is: 3.64


We now construct an interactive plot to study the behaviour of our consumer for different of the various parameters and values

In [10]:
ct_range = np.arange(0, 10, 0.25)
ct1_range = np.arange(0, 10, 0.25)

def _max_problem(w_t, w_t1, theta, rho):
    fig1 = plt.figure(dpi=100)
    ax = fig1.add_subplot(1,1,1)
    
    #grid of (x,y) values which we will pass to function
    ct_range = np.arange(0, 10, 0.25)
    ct1_range = np.arange(0, 10, 0.25)
    c_t, c_t1 = np.meshgrid(ct_range, ct1_range)

    #Plot utility
    ut = inter_util(c_t, c_t1)

    #plot the budget constraint
    cons_constraint = np.linspace(0, 20, 50)
    ax.plot(cons_constraint, (1 + r) * (w_t - cons_constraint) + w_t1, 
            color='b')
    
    #plot the indifference curve
    CS = ax.contour(c_t, c_t1, ut, np.array([utility_star]), colors='black', linewidths=1, linestyles='solid')
    ax.clabel(CS, inline=1, fmt='%1.4f')

    #show the optimal bundle
    ax.hlines(ct_star, 0, ct1_star, linestyle='dashed')
    ax.vlines(ct1_star, 0, ct_star, linestyle='dashed')
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.set_xlabel(r'Present consumption, $c_t$', fontsize = 10)
    ax.set_ylabel(r'Future Consumption, $c_{t+1}$', fontsize = 10)
    ax.set_title(r'Optimal bundle in the case of a CRRA function', fontsize=15)
    
def plot_time():
    widgets.interact(_max_problem,
    w_t= widgets.FloatSlider(
           description='$w_{t}$',
           min=0,
           max=10,
           step=1,
           value=5,
           continuous_update=False,
    ),
    
    w_t1 = widgets.FloatSlider(
            description="$w_{t1}$",
            min=0,
            max=10,
            step=1,
            value=5,
            continuous_update=False,
    ),
    theta = widgets.FloatSlider(
            description="$\\theta$",
            min=0.5,
            max=0.9,
            step=0.02,
            value=0.5,
            continuous_update=False,
    ),
    rho = widgets.FloatSlider(
            description="$\\rho$",
            min=0.9,
            max=0.99,
            step=0.01,
            value=0.9,
            continuous_update=False,
    ),
);  
    

In [11]:
plot_time()


interactive(children=(FloatSlider(value=5.0, continuous_update=False, description='$w_{t}$', max=10.0, step=1.…

## 2. Model description

One way to construct a theory of endogenous growth is to eliminate the long-run tendency for capital to experience diminishing returns. We consider the standard AK model, in which the returns to capital are always constant. We begin our analysis by combining the AK technology with optimizing behavior of households and firms.


In [6]:
c = sm.symbols('c')
alpha = sm.symbols('alpha')
theta = sm.symbols('theta')
e = sm.symbols('e')
rho = sm.symbols('rho') #discount factor
n = sm.symbols('n')
r = sm.symbols('r')
w = sm.symbols('w')
a=sm.symbols('a')
t= sm.symbols('t')
u= sm.symbols('u')
A= sm.symbols('A')
k= sm.symbols('k')
K=sm.symbols('K')
L=sm.symbols('L')
delta = sm.symbols('delta')

### Behavior of household

We identify the most important variables:

- *c* is the consumption
- $\alpha$ is the assets per person
- *r* is the interest rate w is the wage rate
- *w* is the wage rate
- *n* is the growth rate of population

Infinite-lived households maximize utility as given by

$$U= \int_{0}^{\infty} e^{-(\rho-n)t}\left[\frac{c^{1-\theta}-1}{1-\theta}\right]dt$$

subject to constraint

$$\dot{\alpha} = (r-n)\alpha+w-c$$

We impose the no Ponzi-game condition (ruling out chain-letter debt finance)

$$\lim_{t \to \infty} \bigg\{ \alpha(t)exp\left[-\int_{0}^{t}[r(v)-n]dv\right] \bigg\} \geq 0$$

The condition for optimization of the above problem is

$$\frac{\dot{c}}{c}=\frac{1}{\theta}(r-\rho)$$

And the transversality condition

$$\lim_{t \to \infty} \bigg\{ \alpha(t)exp\left[-\int_{0}^{t}[r(v)-n]dv \right] \bigg\} = 0$$

We can use sympy to find the  analytical expression for the optimal value:

In [7]:
def integrand(t, rho, theta, n, c):
    return sm.exp(-t*(rho-n))*((c**(1-theta)-1)/(1-theta))

def expint(rho, theta, n, c):
    return quad(integrand, 0, np.inf, args=(rho, theta, n, c))[0]

def ut(c):
    if theta != 1:
        return (c**(1-theta)-1)/(1-theta)
    else:
        return np.log(c)

In [10]:
expint(0.9, 0.2, 0, 4)

2.8214349069733284

In [20]:
c1 = sm.symbols('c1')
c2 = sm.symbols('c2')
def U(c1, c2):
    return ut(c1) + np.exp(-rho) * ut(c2)


## Behaviour of firm

Our firm has a linear production function:
$$y=f(k)=Ak$$
where A>0. 

We now check that in this case the marginal product of capital is not diminishing ($f''=0$) and that the Inada conditions are violated because $lim_{t \to \infty} [f'(k)] \neq 0$.

In [18]:
# Production function
prod = A*k
deriv= Derivative(prod, k)
foc = sm.Eq(sm.diff(prod,k)
print(f'The first derivative is:' foc)

# Define profit function
profit = prod - (r+delta)*K -w*L
display(profit)

SyntaxError: invalid syntax (<ipython-input-18-d2755546e290>, line 5)

The condition for profit maximization require the marginal product of capital to equal the rental price where the first is $A$ as previously computed. So the condition for profit maximization is:

In [97]:
foc1 = sm.Eq(sm.diff(profit,K),0)
display(foc1)

Eq(A - delta - r, 0)

## Model equilibrium

From profit maximization, we can impose the equilibrium condition for the optimal consumption path, i.e. $r(t) = A-\delta$:
$$\gamma= \frac{\dot{c}(t)}{c(t)}= \sigma(A-\delta-\rho)$$
We can easily notice that for the consumption growth to be positive $A> \delta+\rho$.

In [33]:
# Check if consumption growth is positive

def negpos(A, delta, rho):
    if A > delta + rho:
       print(f'Consumption growth is positive')
    else:
       print(f'Consumption growth is negative')

negpos(1, 2, 1)

Consumption growth is negative


We now solve the differantial equation for the optimal consumption path. The result is:

$$c(t)=c(0)e^{[A-\delta-\rho]t}$$
where c(0) is consumption at time $t_0$.  

In [ ]:
# Solving differential equation for multiple values
A =2 
def model(c, t, sigma, delta, rho):
    dcdt = sigma**(A-delta-rho)
    return dcdt

#initial condition
c0 = 3

#time point
t=np.linspace(0,10,10) #start,f

# Solve ODEs
sigma= 0.1
delta= 0.3
rho= 0.9
c1 = odeint(model,c0,t, args=(sigma, delta, rho,))
sigma=0.2
delta=0.3
rho=0.9
c2 = odeint(model,c0,t, args=(sigma, delta, rho,))
sigma= 0.3
delta= 0.3
rho= 0.9
c3 = odeint(model,c0,t, args=(sigma, delta, rho,))

#plot results
plt.plot(t,c1,'r-', linewidth=2)
plt.plot(t,c2,'b--', linewidth=2)
plt.plot(t,c3,'g:', linewidth=2)
plt.xlabel('time')
plt.ylabel('c(t)')

Another important condition is to ensure that $\delta +\rho> (A-\delta)(1-\theta)+\theta*n+\delta$. In this way it is ensured that the attainable utility is bounded and the transversality condition holds.

In [35]:
#Check if utility is bounded

def to_bounded(delta, rho, A, theta, n):
    if delta+rho > (A-delta)*(1-theta)+theta*n+delta:
        print(f'Utility is bounded')
    else:
        print(f'Utility is not bounded')

to_bounded(1, 1, 2, 1, 3)

Moreover, in a closed economy, domestic production equals domestic demand (investment plus consumption) and capital is owned by residents implying $\alpha=k$.

To compute the growth rate of capital and output per worker, we divide eq.1 by k to get:
$$\frac{c}{k}=(A-\delta-n)-\frac{\dot{k}}{k}$$
In the steady state the groth rate of capital per person is constant. Therefore, the right-hand sideof the expression for $c/k$ is constant. Consequently th growth rate of capital per person (and hence the growth rate of output per person, $y$) equals the growth rate of consumption per capita already defined.

In [ ]:
# note: documentation not written yet

import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams["axes.prop_cycle"]
colors = prop_cycle.by_key()["color"]
import ipywidgets as widgets

class AkModel:

    ############
    # 1. setup #
    ############

    def __init__(self, name="", **kwargs):

        self.name = name
        self.parameters()
        self.update_parameters(kwargs)
        self.primitive_functions()

    def baseline_parameters(self):

        # a. model parameters
        self.rho = 0.90  # discount factor
        self.n = 0 # growth of population

        self.ut_function = "crra"
        self.theta = 0.5
        self.sigma = 1/self.theta  # crra coefficient

        self.production_function = "Ak"
        self.A = 2 # constant
        self.delta = 0.5  # depreciation rate

        # b. solution
        self.tau = 1e-7  # tolerance

        # c. saddle-path
        self.k0 = 1.00  # initial level of capital
        self.saddlepath_max_iter = 1000  # maximum number of iterations

        # d. loci in phase diagram
        self.Nk = 200  # num of points
        self.k_max = 10  # maximum capital
        self.c_max = 10  # maximum consumption
        
        # e. simulation
        self.T = 500  # number of periods
        self.c_upper_fac = (
            5
        )  # exit if consumption > c_upper_fac*consumption in steady state
        self.c_lower_fac = (
            0.05
        )  # exit if consumption < c_lower_fac*consumption in steady state
        self.force_steady_state = 1e-4  # assume in steady state if close enough

        # f. misc
        self.do_print = True

    def update_parameters(self, kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

    def primitive_functions(self):

        eps = 1e-12  # minimum evaluation value to avoid errors

        # a. utility function
        if self.ut_function == "crra":
            if self.theta == 1:
                self.u = lambda c: np.log(np.fmax(c, eps))
            else:
                self.u = lambda c: (np.fmax(c, eps) ** (1 - self.theta) -1) / (
                    1 - self.theta
                )
            self.u_prime = lambda c: np.fmax(c, eps) ** (-self.theta)
        else:
            raise ValueError("unknown utility function")

        # b. production function
        if self.production_function == "Ak":
            self.f = lambda k: np.fmax(k, eps) * self.A
            self.f_prime = lambda k: self.A 
        else:
            raise ValueError("unknown production function")
        
        

    
    ######################
    # 2. model functions #
    ######################

    def k_plus_func(self, k, c):  # next period capital
        return k * (1 - self.delta) + self.f(k) - c

    def c_func(self, k, k_plus):  # backwards calculation of consumption
        return k * (1 - self.delta) + self.f(k) - k_plus

    def euler_LHS_func(self, c, c_plus):  # left-hand-side of euler equation
        return self.u_prime(c) / self.u_prime(c_plus)

    def euler_RHS_func(self, k_plus):  # right-hand-side of euler equation
        return self.rho * (1 + self.f_prime(k_plus) - self.delta)

    def r_func(self, k):  # interest rate
        return self.f_prime(k) - self.delta
    
    def model(self)
        return 1/self.theta(self.A - self.delta - self.rho)
    
    def Gamma(self, k, c):  # law-of-motion

        # i. capital
        k_plus = self.k_plus_func(k, c)

        # ii. consumption
        
        obj = lambda c_plus: self.euler_LHS_func(c, c_plus) - self.euler_RHS_func(
            k_plus
        )
        c_plus = optimize.fsolve(obj, c)[0]

        return k_plus, c_plus
    
    ############
    # 3. solve #
    ############
    def find_steady_state(self):

        # a. objective euler_RHS_func == 1
        obj = lambda k: self.euler_RHS_func(k) - 1

        # b. find capital in steady state
        self.k_star = optimize.fsolve(obj, 1)[0]

        # c. find implied steady consumption, interest rate and wage
        self.c_star = self.c_func(self.k_star, self.k_star)
        self.r_star = self.r_func(self.k_star)
        self.w_star = self.w_func(self.k_star)

        # d. print
        if self.do_print:
            print(f"k_star = {self.k_star:.4f}")
            print(f"c_star = {self.c_star:.4f}")
            print(f"r_star = {self.r_star:.4f}")
            print(f"w_star = {self.w_star:.4f}")
    
    def find_c0_on_saddlepath(self):  # initial consumption

        # a. initial capital
        k0 = self.k0
        
        # b. initial interval for consumption
        c0 = self.c_func(k0, k0)

        # for plotting
        self.c0_path = np.nan * np.ones(self.saddlepath_max_iter)

        # c. algorithm
        t = 0  # time
        it = 0  # iteration counter
        while it < self.saddlepath_max_iter:

            # i. initialize
            if t == 0:

                k = k0
                c = c0

                # for plotting
                self.c0_path[it] = c0

                it += 1

            t += 1  # increment time

            # ii. update and distance
            k, c = self.Gamma(k, c)
            dist = self.dist_to_ss(k, c)

            # iii. finish, forward or restart?
            if dist < self.tau or t > self.saddlepath_T:  # finish
                break
            else:
                continue
        

        self.k0, self.c0 = k0, c0
    
    def calculate_loci(self):

        # a. setup
        self.k_loci = dict()

        # c. k loci
        self.k_loci["x"] = np.linspace(0, self.k_max, self.Nk)
        self.k_loci["y"] = self.c_func(self.k_loci["x"], self.k_loci["x"])

    
    ###############
    # 4. simulate #
    ###############
    
    def simulate(self, c0=None):

        # a. allocate
        self.sim = dict()
        self.sim["k"] = np.empty(self.T)
        self.sim["c"] = np.empty(self.T)
        self.sim["r"] = np.empty(self.T)
        self.sim["w"] = np.empty(self.T)

        # b. time loop
        for t in range(self.T):

            # i. initial values
            if t == 0:

                self.sim["k"][t] = self.k0
                if c0 == None:
                    self.sim["c"][t] = self.c0  # model state
                else:
                    self.sim["c"][t] = c0  # input

            # ii. forward
            else:

                if (
                    self.dist_to_steady(self.sim["k"][t - 1], self.sim["c"][t - 1])
                    < self.force_steady_state
                ):
                    self.sim["k"][t] = self.k_star
                    self.sim["c"][t] = self.c_star
                else:
                    self.sim["k"][t], self.sim["c"][t] = self.Gamma(
                        self.sim["k"][t - 1], self.sim["c"][t - 1]
                    )

            # iii. prices
            self.sim["r"][t] = self.r_func(self.sim["k"][t])
            self.sim["w"][t] = self.w_func(self.sim["k"][t])

            # iv. disconvergence
            if (
                self.sim["c"][t] > self.c_upper_fac * self.c_star
                or self.sim["c"][t] < self.c_lower_fac * self.c_star
            ):
                self.sim["k"][t:] = np.nan
                self.sim["c"][t:] = np.nan
                break

    def optimal_initial_consumption_level(self):

        r_path = self.sim["r"]
        w_path = self.sim["w"]

        # a. initialize sums
        c_sum_rel = 1  # discounted sum of consumption relative to c0
        wealth_sum = (1 + r_path[0]) * self.k0 + w_path[0]  # discounted wealth

        # b. initialize factors
        c_fac = np.ones(self.T)  # growth factor of consumption
        R_fac = 1  # total discount factor

        # c. calculate sums
        for t in range(1, self.T):  # t = 1,2,3,...

            # i. factors
            c_fac[t] = (
                c_fac[t - 1]
                * ((1 + r_path[t]) * self.rho) ** (1 / self.sigma)
                / (1 + r_path[t])
            )
            R_fac /= 1 + r_path[t]

            # ii. sums
            c_sum_rel += c_fac[t]
            wealth_sum += w_path[t] * R_fac

        # d. return optimal consumption choice
        c0 = wealth_sum / c_sum_rel
        return c0 
                  
    ###########
    # 5. plot #
    ###########

    def plot_loci(self, ax, **kwargs):
        self.calculate_loci()
        self.plot_k_loci(ax, **kwargs)
        ax.set_xlim([0, self.k_max])
        ax.set_ylim([0, self.c_max])
        ax.set_xlabel("$k_t$")
        ax.set_ylabel("$c_t$")

    def plot_k_loci(self, ax, **kwargs):
        ax.plot(self.k_loci["x"], self.k_loci["y"], color="red", **kwargs)

    def plot_steady_state(self, ax, **kwargs):
        ax.plot(self.k_star, self.c_star, **kwargs)

    def plot_kc_path(self, ax, **kwargs):
        ax.plot(self.sim["k"], self.sim["c"], "-o", lw=0.5, MarkerSize=2, **kwargs)

    def plot_sim_time(self, ax, varname, **kwargs):
        ax.plot(self.sim[varname], "o", MarkerSize=2, **kwargs)



#####################
# interactive plots #
#####################

def interactive_phase_diagram():

    widgets.interact(
        phase_diagram,
        rho=widgets.FloatSlider(
            description="$\\rho$",
            min=0.90,
            max=0.99,
            step=0.01,
            value=0.90,
            continuous_update=False,
        ),
        sigma=widgets.FloatSlider(
            description="$\\theta$",
            min=0.5,
            max=4,
            step=0.01,
            value=0.5,
            continuous_update=False,
        ),
        delta=widgets.FloatSlider(
            description="$\\delta$",
            min=0.05,
            max=0.20,
            step=0.01,
            value=0.5,
            continuous_update=False,
        ),
        A=widgets.FloatSlider(
            description="$A$",
            min=0.5,
            max=7,
            step=0.5,
            value=2,
            continuous_update=False,
        ),
        k0=widgets.FloatSlider(
            description="$k_0$",
            min=0,
            max=10,
            step=0.05,
            value=1,
            continuous_update=False,
        ),
    )

def phase_diagram(rho, sigma, delta, A, k0):

    # a. solve model
    model = AkModel(
        rho=rho, theta=theta, delta=delta, A=A, k0=k0, do_print=False
    )
    model.find_steady_state()
    model.find_c0_on_saddlepath()

    # b. setup figure
    fig = plt.figure(figsize=(12, 4), dpi=100)

    # c. phase diagram
    ax = fig.add_subplot(1, 2, 1)
    model.plot_loci(ax)
    model.simulate()
    model.plot_kc_path(ax)


In [ ]:
interactive_phase_diagram()